In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/Colab Notebooks/Datasets/MahineHack - Retail Price Prediction'

 DataPrep1.csv	   'Sample Submission.csv'   Train.csv
 predictions1.csv   Test.csv


In [ ]:
Data_dir = '/content/gdrive/My Drive/Colab Notebooks/Datasets/MahineHack - Retail Price Prediction/'
df = pd.read_csv(Data_dir+ 'DataPrep1.csv')
test_data = pd.read_csv(Data_dir + 'Test.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,Month,Day,Timelight,Weekday
0,6141,1583,144,3,3.75,14056.0,35,2011,5,6,Noon,Friday
1,6349,1300,3682,6,1.95,13098.0,35,2011,5,11,Morning,Wednesday
2,16783,2178,1939,4,5.95,15044.0,35,2011,11,20,Noon,Sunday
3,16971,2115,2983,1,0.83,15525.0,35,2011,11,22,Noon,Tuesday
4,6080,1210,2886,12,1.65,13952.0,35,2011,5,6,Morning,Friday


In [ ]:
df[df.UnitPrice==0].head(2)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,Month,Day,Timelight,Weekday
15291,15305,3681,2140,1,0.0,12437.0,13,2011,C,week1,Noon,Thursday
26477,17360,3034,248,12540,0.0,13256.0,35,2011,C,week3,Noon,Friday


## Transformation

In [ ]:
def transformation(data):
  data['Quantity'] = data['Quantity'].abs()
  data['Month'] = data['Month'].apply(lambda x: 'A' if x in range(1,5) else ( 'B' if x in range(5,9) else  'C'))
  data['Day'] = data['Day'].apply(lambda x: 'week1' if x in range(1,11) else ( 'week2' if x in range(11,21) else 'week3') )
  data1 = data.drop(['InvoiceNo'],axis=1)
  return data1

In [ ]:
df1 = transformation(df)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   StockCode    284780 non-null  int64  
 1   Description  284780 non-null  int64  
 2   Quantity     284780 non-null  int64  
 3   UnitPrice    284780 non-null  float64
 4   CustomerID   284780 non-null  float64
 5   Country      284780 non-null  int64  
 6   Year         284780 non-null  int64  
 7   Month        284780 non-null  object 
 8   Day          284780 non-null  object 
 9   Timelight    284780 non-null  object 
 10  Weekday      284780 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 23.9+ MB


In [ ]:
for i in range(0,len(df1)):
  if df1.loc[i,'UnitPrice'] == 0 :
    a = df1.loc[i,'StockCode']
    df1.loc[i,'UnitPrice'] = df1[df1.StockCode == a].UnitPrice.mean()

In [ ]:
df1[df1.StockCode==3681].UnitPrice.mean()

339.938574117959

In [ ]:
#df[df.StockCode==df['StockCode'][0]].Description.value_counts().index.tolist()[0]
#df['StockCode'][0]

"""for i in range(0,len(df)):
   a = df.loc[i,'StockCode']
   b = df[df.StockCode==a].Description.value_counts().index.tolist()[0]
   if df.loc[i,'Description']==b:
     df.loc[i,'Dcode'] = 'Common'
   else :
     df.loc[i,'Dcode'] = 'Uncommon'
df.head()"""

df1.head()

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,Month,Day,Timelight,Weekday
0,1583,144,3,3.75,14056.0,35,2011,B,week1,Noon,Friday
1,1300,3682,6,1.95,13098.0,35,2011,B,week2,Morning,Wednesday
2,2178,1939,4,5.95,15044.0,35,2011,C,week2,Noon,Sunday
3,2115,2983,1,0.83,15525.0,35,2011,C,week3,Noon,Tuesday
4,1210,2886,12,1.65,13952.0,35,2011,B,week1,Morning,Friday


In [ ]:
df.Dcode.value_counts()

Common      280736
Uncommon      4044
Name: Dcode, dtype: int64

In [ ]:
X = df[['StockCode', 'Quantity', 'Country','Year','Month','Day','Weekday','Timelight','Description','CustomerID']]
y = df['UnitPrice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=None)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


num_attribs = ['StockCode','Quantity','Description','CustomerID']
cat_attribs = [ 'Country','Weekday','Timelight','Year','Month','Day']

full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])



In [ ]:
X_train  = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.fit_transform(X_test)
X_train.shape

(227824, 58)

In [ ]:
X_test.shape

(56956, 58)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
#from sklearn.svm import SVR
#from sklearn.linear_model import LinearRegression

rfr = RandomForestRegressor(max_depth=20, random_state=23)
#svr = SVR(C=1.0, epsilon=0.2)
#lr = LinearRegression()

rfr.fit(X_train,y_train)
#svr.fit(X_train,y_train)
#lr.fit(X_train,y_train)

pred_rfr = rfr.predict(X_test)
#pred_svr = svr.predict(X_test)
#pred_lr = lr.predict(X_test)

print('The error in Random forest Regression is: '+ str(np.sqrt(mean_squared_error(y_test, pred_rfr))))
#print('The error in Support Vector Regression is: '+ str(np.sqrt(mean_squared_error(y_test, pred_svr))))
#print('The error in Linear regression is: '+ str(np.sqrt(mean_squared_error(y_test, pred_lr))))


The error in Random forest Regression is: 26.562647844003806


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=55)
tree_reg.fit(X_train, y_train)

predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

19.8941521387818

In [ ]:
def test_data_transformation(train):
  train['Date'] = train['InvoiceDate'].apply(lambda x: x.split(' ')[0])
  train['InvoiceDate'] = pd.to_datetime(train['InvoiceDate'])
  train['Year'] = train['InvoiceDate'].dt.year
  train['Month'] = train['InvoiceDate'].dt.month
  train['Day'] = train['InvoiceDate'].dt.day
  train['Time'] = train['InvoiceDate'].dt.time 
  train['Timelight'] = train['Time'].apply(lambda x : 'Morning' if 6<=x.hour<12 else ('Noon' if 12<=x.hour<18 else 'Night' ))
  day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
  train['Weekday'] = train['Date'].apply(lambda x: day_name[datetime.datetime.strptime(x, '%Y-%m-%d').weekday()])
  #train1 = train.copy()
  train.drop(['InvoiceDate','Time','InvoiceNo'],axis = 1,inplace=True)
  train['Quantity'] = train['Quantity'].abs()
  train['Month'] = train['Month'].apply(lambda x: 'A' if x in range(1,5) else ( 'B' if x in range(5,9) else  'C'))
  train['Day'] = train['Day'].apply(lambda x: 'week1' if x in range(1,11) else ( 'week2' if x in range(11,21) else 'week3') )
  return train



In [ ]:
test_data_transformation(test_data)
test_data.head()

,StockCode,Description,Quantity,CustomerID,Country,Date,Year,Month,Day,Timelight,Weekday
0,1709,1079,3,16161.0,35,2011-02-22,2011,A,week3,Noon,Tuesday
1,510,3457,1,17341.0,35,2010-12-08,2010,C,week1,Noon,Wednesday
2,604,694,36,15158.0,35,2011-10-25,2011,C,week3,Noon,Tuesday
3,1478,3473,2,16033.0,35,2011-06-27,2011,B,week3,Noon,Monday
4,3216,871,1,15351.0,35,2011-11-06,2011,C,week1,Noon,Sunday


In [ ]:
test_data.drop(['Date'],axis=1,inplace=True)

In [ ]:
print(test_data.shape)
test_data = full_pipeline.fit_transform(test_data)
print(test_data.shape)

(122049, 10)
(122049, 58)


In [ ]:
def test_submission(submission_number,model,test_data):
    predictions = model.predict(test_data)
    np.savetxt(Data_dir+"predictions"+str(submission_number)+".csv", predictions, delimiter=",")
    
test_submission(7,rfr,test_data)

### Implementing Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
print(X.shape)
print(y.shape)
X  = full_pipeline.fit_transform(X)
print(X.shape)

(284780, 10)
(284780,)
(284780, 58)


In [ ]:
X_train = X_train.toarray()
X_test = X_test.toarray()

#X_train = torch.from_numpy(X_train).to_sparse()
#X_test = torch.from_numpy(X_test).to_sparse()
type(X_train)

numpy.ndarray

In [ ]:
import tensorflow as tf
import torch

#X_train = tf.sparse.to_dense(X_train, default_value=None, validate_indices=True, name=None)
#X_test = tf.sparse.to_dense(X_test, default_value=None, validate_indices=True, name=None)
model = Sequential()
model.add(Dense(58, input_dim=58, kernel_initializer='normal', activation='relu'))
model.add(Dense(29, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))

# Compile model

model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=150,batch_size=32)

predictions = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, predictions))

Epoch 1/150
7120/7120 [==============================] - 8s 1ms/step - loss: 7275.5283 - accuracy: 2.5458e-04
Epoch 2/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7261.2515 - accuracy: 2.4580e-04
Epoch 3/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7241.0156 - accuracy: 2.4580e-04
Epoch 4/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7206.7256 - accuracy: 2.5019e-04
Epoch 5/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7173.5767 - accuracy: 2.3703e-04
Epoch 6/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7159.2007 - accuracy: 2.4580e-04
Epoch 7/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7143.2812 - accuracy: 2.4580e-04
Epoch 8/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7117.0205 - accuracy: 2.5019e-04
Epoch 9/150
7120/7120 [==============================] - 9s 1ms/step - loss: 7093.9771 - accuracy: 2.5458e-04
Epoch 10/1

41.16987892027163

In [ ]:
sample_X = sample_X.toarray()
type(sample_X)

numpy.ndarray

In [ ]:
def larger_model():
 # create model
 km = Sequential()
 km.add(Dense(58, input_dim=58, kernel_initializer='normal', activation='relu'))
 km.add(Dense(35, kernel_initializer='normal', activation='relu'))
 km.add(Dense(25, kernel_initializer='normal', activation='relu'))
 km.add(Dense(1, kernel_initializer='normal'))
 # Compile model
 km.compile(loss='mean_squared_error', optimizer='adam')
 return km

# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=5)
results = cross_val_score(estimator, sample_X, sample_y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Baseline: -50974.74 (101917.86) MSE


In [ ]:
estimator.fit(sample_X, sample_y)
predictions = estimator.predict(X_test)
print(np.sqrt(mean_squared_error(y_test, predictions)))

24.932249750546838


Sampling Techniques

In [ ]:
#https://www.kaggle.com/flaviobossolan/stratified-sampling-python
# the functions:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df



def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [ ]:
help(stratified_sample_report)

Help on function stratified_sample_report in module __main__:

stratified_sample_report(df, strata, size=None)
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of t

In [ ]:
sample_df = stratified_sample(df1, ['StockCode'],size=40000,seed=163, keep_index= True)
print(len(sample_df))
sample_df.tail()

39968


,index,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,Month,Day,Timelight,Weekday
39963,255581,3683,2624,1,18.0,12708.0,14,2011,C,week1,Morning,Thursday
39964,248957,3683,2624,3,18.0,12678.0,13,2011,C,week3,Morning,Thursday
39965,140275,3683,2624,3,28.0,12545.0,30,2011,A,week3,Noon,Friday
39966,126428,3683,2624,3,15.0,13023.0,35,2011,B,week1,Morning,Tuesday
39967,129447,3683,2624,3,40.0,13520.0,32,2011,B,week1,Morning,Wednesday


In [ ]:
sample_X = sample_df[['StockCode', 'Quantity', 'Country','Year','Month','Day','Weekday','Timelight','Description','CustomerID']]
sample_y = sample_df['UnitPrice']
sample_X = full_pipeline.fit_transform(sample_X)

In [ ]:
from sklearn.model_selection import train_test_split
Xs_train, Xs_test, ys_train, ys_test = train_test_split(sample_X,sample_y, test_size=0.25, random_state=None)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

rfr = RandomForestRegressor(max_depth=20, random_state=23)
#svr = SVR(C=1.0, epsilon=0.2)
#lr = LinearRegression()

rfr.fit(Xs_train,ys_train)
#svr.fit(sample_X,sample_y)
##lr.fit(sample_X,sample_y)

pred_rfr = rfr.predict(Xs_test)
#pred_svr = svr.predict(X_test)
#pred_lr = lr.predict(X_test)

print('The error in Random forest Regression is: '+ str(np.sqrt(mean_squared_error(ys_test, pred_rfr))))
#print('The error in Support Vector Regression is: '+ str(np.sqrt(mean_squared_error(y_test, pred_svr))))
#print('The error in Linear regression is: '+ str(np.sqrt(mean_squared_error(y_test, pred_lr))))

The error in Random forest Regression is: 16.92891579533915


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=55)
tree_reg.fit(sample_X,sample_y)

predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

17.393460031836163

In [ ]:
def test_submission(submission_number,model,test_data):
    predictions = model.predict(test_data)
    np.savetxt(Data_dir+"predictions"+str(submission_number)+".csv", predictions, delimiter=",")
    
#test_submission(7,rfr,test_data)
test_submission(8,svr,test_data)

In [ ]:
test_submission(9,estimator,test_data)

In [ ]:
sample_df.to_csv(Data_dir+'sample_Data.csv',index=False)

In [ ]:
"Link for next Notebook - https://colab.research.google.com/drive/1DmWP2UvfxRn3LUzOQoj3XDfPvXy-emPa?usp=sharing"

'Link for next Notebook - https://colab.research.google.com/drive/1DmWP2UvfxRn3LUzOQoj3XDfPvXy-emPa?usp=sharing'

In [ ]:
stratified_sample_report(df1, ['StockCode'],size=40000)[stratified_sample_report(df1, ['StockCode'],size=40000)['samp_size']==0]

,StockCode,size,samp_size
3,3,3,0
5,5,3,0
26,26,2,0
32,32,2,0
33,33,2,0
...,...,...,...
3606,3670,1,0
3607,3672,2,0
3608,3673,1,0
3609,3674,2,0
